In [1]:
from datetime import datetime
from qiskit.providers.models import BackendProperties
from qiskit.converters import circuit_to_dag, dag_to_circuit
from qiskit import Aer, IBMQ, execute
from qcg.generators import gen_supremacy, gen_hwea
from helper_fun import simulate_circ, find_saturated_shots, load_IBMQ, cross_entropy
from qiskit.providers.aer import noise
from qiskit.transpiler.passes import NoiseAdaptiveLayout
import MIQCP_searcher as searcher
import cutter
from evaluator_prob import find_rank_combinations, evaluate_cluster
from uniter_prob import reconstruct
from time import time

provider = load_IBMQ()
public_provider = IBMQ.get_provider('ibm-q')
mel = public_provider.get_backend('ibmq_16_melbourne')
prop = mel.properties()
qubit_list = prop.qubits[:]
for i in range(len(qubit_list)):
    idx = -1
    for j,nduv in enumerate(qubit_list[i]):
        if nduv.name == 'readout_error':
            idx = j
            break
    if idx != -1:
        qubit_list[i][idx].value = 0.0
calib_time = datetime(year=2019, month=10, day=15, hour=0, minute=0, second=0) #junk, set any time you like
bprop = BackendProperties(last_update_date=calib_time, backend_name="no_readout_error", qubits=qubit_list, backend_version="1.0.0", gates=prop.gates, general=[])
coupling_map = mel.configuration().coupling_map
noise_model = noise.device.basic_device_noise_model(bprop)
basis_gates = noise_model.basis_gates
case = (2,4)
circ = gen_supremacy(int(case[0]),int(case[1]),8,order='75601234')
saturated_shots = find_saturated_shots(circ)
qasm_info = [mel,prop,bprop,coupling_map,noise_model,basis_gates,saturated_shots]
print(circ)

┌───┐┌───┐      ┌──────────┐   ┌───┐                                 »
q0_0: |0>┤ H ├┤ T ├────■─┤ Ry(pi/2) ├───┤ T ├──────────────────────────■──────»
         ├───┤├───┤    │ └──────────┘┌──┴───┴───┐┌───┐   ┌───┐         │      »
q0_1: |0>┤ H ├┤ T ├─■──■──────■──────┤ Rx(pi/2) ├┤ T ├───┤ H ├─────────┼──────»
         ├───┤├───┤ │         │      └──────────┘└───┘┌──┴───┴───┐     │      »
q0_2: |0>┤ H ├┤ T ├─┼─────────■────────────────────■──┤ Ry(pi/2) ├─────┼──────»
         ├───┤├───┤ │    ┌──────────┐   ┌───┐      │  ├──────────┤     │      »
q0_3: |0>┤ H ├┤ T ├─┼──■─┤ Ry(pi/2) ├───┤ T ├──────■──┤ Ry(pi/2) ├─────┼──────»
         ├───┤├───┤ │  │ └──────────┘   └───┘         ├──────────┤     │      »
q0_4: |0>┤ H ├┤ T ├─┼──┼───────────────────────────■──┤ Rx(pi/2) ├─────■──────»
         ├───┤├───┤ │  │ ┌──────────┐   ┌───┐      │  └──────────┘┌──────────┐»
q0_5: |0>┤ H ├┤ T ├─■──┼─┤ Rx(pi/2) ├───┤ T ├──────■───────■──────┤ Rx(pi/2) ├»
         ├───┤├───┤    │ └──────────┘┌──┴───┴───┐

In [2]:
ground_truth = simulate_circ(circ,'statevector_simulator',None)
qasm = simulate_circ(circ,'noiseless_qasm_simulator',qasm_info)
vanilla_execution = simulate_circ(circ,'noisy_qasm_simulator',qasm_info)

In [3]:
hardness, positions, ancilla, d, num_cluster, m = searcher.find_cuts(circ,num_clusters=range(2,4),hw_max_qubit=4,evaluator_weight=1)
m.print_stat()
clusters, complete_path_map, K, d = cutter.cut_circuit(circ, positions)

evaluator_input = {case:[None,None,None,None,None,clusters,complete_path_map]}
rank = 0
size = 2
num_workers = size - 1
rank_combinations = find_rank_combinations(evaluator_input,rank,size)
rank_results = {}
rank_classical_time = {}
rank_quantum_time = {}
for key in rank_combinations:
    rank_results[key] = {}
    rank_quantum_time[key] = 0
    rank_classical_time[key] = 0
    _,_,_,_,_,clusters,complete_path_map = evaluator_input[key]
    for cluster_idx in range(len(rank_combinations[key])):
        quantum_evaluator_begin = time()
        rank_shots = max(int(saturated_shots/len(rank_combinations[key][cluster_idx])/num_workers)+1,500)
        print('rank {} runs case {}, cluster_{} * {} on QUANTUM, sameTotal shots = {}'.format(
            rank,key,cluster_idx,
            len(rank_combinations[key][cluster_idx]),rank_shots))
        cluster_prob = evaluate_cluster(complete_path_map=complete_path_map,
        cluster_circ=clusters[cluster_idx],
        combinations=rank_combinations[key][cluster_idx],
        backend='noisy_qasm_simulator',num_shots=rank_shots,provider=provider)
        # cluster_prob = evaluate_cluster(complete_path_map=complete_path_map,
        # cluster_circ=clusters[cluster_idx],
        # combinations=rank_combinations[key][cluster_idx],
        # backend='statevector_simulator')
        rank_quantum_time[key] += time()-quantum_evaluator_begin
        rank_results[key][cluster_idx] = cluster_prob

Academic license - for non-commercial use only
solving for 2 clusters
Infeasible
solving for 3 clusters
MIQCP stats:
node count: 192.0
10 vertices 12 edges graph. Max qubit = 4
4 cuts, 3 clusters
cluster 0: original input = 3.00, rho qubits = 1.00, O qubits = 2.00, d = 4.00, K = 3.00
cluster 1: original input = 4.00, rho qubits = 0.00, O qubits = 2.00, d = 4.00, K = 2.00
cluster 2: original input = 1.00, rho qubits = 3.00, O qubits = 0.00, d = 4.00, K = 3.00
objective value: 279.22804751435876
manually calculated objective value: 279.00000000000006
mip gap: 0.0
runtime: 0.12802791595458984
OPTIMAL
rank 0 runs case (2, 4), cluster_0 * 9 on QUANTUM, sameTotal shots = 7112
rank 0 runs case (2, 4), cluster_1 * 54 on QUANTUM, sameTotal shots = 1186
rank 0 runs case (2, 4), cluster_2 * 216 on QUANTUM, sameTotal shots = 500


In [4]:
reconstructed_prob = reconstruct(complete_path_map=complete_path_map, full_circ=circ, cluster_circs=clusters, cluster_sim_probs=rank_results[case])
print(cross_entropy(ground_truth,ground_truth))
print(cross_entropy(ground_truth,vanilla_execution))
print(cross_entropy(ground_truth,reconstructed_prob))

4.940935539597721
5.321878393997983
4.940935539597722
